# Import and load combined csv

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob

%matplotlib inline
plt.rcParams['figure.figsize'] = (40, 20)

In [74]:
sales_data = pd.read_csv('../sydney_sales_combined.csv')
sales_data[41000:41500].head(1)

,Unnamed: 0,ADDRESS,SALE PRICE,SALE DATE,AREA,STRATA/NON STRATA,MULTI-PROPERTY SALE (Y/N),PROPERTY NUMBER,DEALING NUMBER,EXTRACTION DATE,SUBURB,SALE DAY,SALE MONTH,SALE YEAR
41000,45311,"168 RILEY STREET, DARLINGHURST NSW 2010",1600000,2012-10-05,113.8,NON STRATA,No,2143084,AH521492,30 April 2017,Darlinghurst,5,10,2012


In [75]:
sales_data.to_csv("../sydney_sales_combined_geocoded.csv")
sales_data = pd.read_csv("../sydney_sales_combined_geocoded.csv")
sales_data[41000:41500].head(1)

,Unnamed: 0,Unnamed: 0.1,ADDRESS,SALE PRICE,SALE DATE,AREA,STRATA/NON STRATA,MULTI-PROPERTY SALE (Y/N),PROPERTY NUMBER,DEALING NUMBER,EXTRACTION DATE,SUBURB,SALE DAY,SALE MONTH,SALE YEAR
41000,41000,45311,"168 RILEY STREET, DARLINGHURST NSW 2010",1600000,2012-10-05,113.8,NON STRATA,No,2143084,AH521492,30 April 2017,Darlinghurst,5,10,2012


# Load test CSV

In [12]:
test_csv = pd.read_csv('../test_suburbs.csv')
test_csv.head()

,Unnamed: 0,ADDRESS,SALE PRICE,SALE DATE,AREA,STRATA/NON STRATA,MULTI-PROPERTY SALE (Y/N),PROPERTY NUMBER,DEALING NUMBER,EXTRACTION DATE,SUBURB,SALE DAY,SALE MONTH,SALE YEA
0,46290,"24/20 MCLACHLAN AVENUE, DARLINGHURST NSW 2010",525000,2012-06-27,0.0,STRATA,No,3850267,AJ732950,30 April 2017,Darlinghurst,27,6,2012
1,46291,"44/20 MCLACHLAN AVENUE, DARLINGHURST NSW 2010",580000,2012-06-19,0.0,STRATA,No,3850283,AJ738253,30 April 2017,Darlinghurst,19,6,2012
2,46292,"310/2 NEILD AVENUE, DARLINGHURST NSW 2010",520000,2012-05-21,0.0,STRATA,No,3850198,AJ749790,30 April 2017,Darlinghurst,21,5,2012
3,46293,"318/4 NEILD AVENUE, DARLINGHURST NSW 2010",595000,2012-05-21,0.0,STRATA,No,3850206,AJ749791,30 April 2017,Darlinghurst,21,5,2012
4,46294,"48/20 MCLACHLAN AVENUE, DARLINGHURST NSW 2010",715000,2012-05-08,0.0,STRATA,No,3850287,AJ790936,30 April 2017,Darlinghurst,8,5,2012


# Geocode something

In [17]:
import sys
sys.path.insert(0, '../python')
import geocode

In [18]:
address = test_csv["ADDRESS"][0]

geo_example = geocode.geocode(address)
print geo_example

{u'status': u'OK', u'results': [{u'geometry': {u'location': {u'lat': -33.8771518, u'lng': 151.2282027}, u'viewport': {u'northeast': {u'lat': -33.8758028197085, u'lng': 151.2295516802915}, u'southwest': {u'lat': -33.8785007802915, u'lng': 151.2268537197085}}, u'location_type': u'ROOFTOP'}, u'formatted_address': u'24/20 McLachlan Ave, Darlinghurst NSW 2010, Australia', u'place_id': u'EjUyNC8yMCBNY0xhY2hsYW4gQXZlLCBEYXJsaW5naHVyc3QgTlNXIDIwMTAsIEF1c3RyYWxpYQ', u'address_components': [{u'long_name': u'24', u'types': [u'subpremise'], u'short_name': u'24'}, {u'long_name': u'20', u'types': [u'street_number'], u'short_name': u'20'}, {u'long_name': u'McLachlan Avenue', u'types': [u'route'], u'short_name': u'McLachlan Ave'}, {u'long_name': u'Darlinghurst', u'types': [u'locality', u'political'], u'short_name': u'Darlinghurst'}, {u'long_name': u'Council of the City of Sydney', u'types': [u'administrative_area_level_2', u'political'], u'short_name': u'Sydney'}, {u'long_name': u'New South Wales', u'

# Try apply method

In [29]:
row_num = 0

def geocode_save(row):
    
    address = row[1]
    
    geo_address = geocode.geocode(address)
    
    location = geo_address["results"][0]["geometry"]["location"]
    council = geo_address["results"][0]["address_components"][3]
    viewport = geo_address["results"][0]["geometry"]["viewport"]

    results = {"lat": location["lat"],
               "lng": location["lng"],
               "council_short": council["short_name"],
               "council_full": council["long_name"],
               "gmaps_id": geo_address["results"][0]["place_id"],
               "view NE lat": viewport["northeast"]["lat"],
               "view NE lng": viewport["northeast"]["lng"],
               "view SW lat": viewport["southwest"]["lat"],
               "view SW lng": viewport["southwest"]["lng"],
               "formatted address": geo_address["results"][0]["formatted_address"],
               "location type": geo_address["results"][0]["geometry"]["location_type"]
                }

    results_keys = results.keys()

    def getter(y):
        return results[y]
    
    for x in results_keys:
        test_csv[x][row_num] = getter(x)
    
    row_num = row_num + 1

In [30]:
test_csv.apply(geocode_save, axis=1)

test_csv.head()

UnboundLocalError: ("local variable 'row_num' referenced before assignment", u'occurred at index 0')

# Try for loop method

In [33]:
new_df = []

def geocode_save(row):
    
    address = row[1]
    
    geo_address = geocode.geocode(address)
    
    location = geo_address["results"][0]["geometry"]["location"]
    council = geo_address["results"][0]["address_components"][3]
    viewport = geo_address["results"][0]["geometry"]["viewport"]

    results = {"lat": location["lat"],
               "lng": location["lng"],
               "council_short": council["short_name"],
               "council_full": council["long_name"],
               "gmaps_id": geo_address["results"][0]["place_id"],
               "view NE lat": viewport["northeast"]["lat"],
               "view NE lng": viewport["northeast"]["lng"],
               "view SW lat": viewport["southwest"]["lat"],
               "view SW lng": viewport["southwest"]["lng"],
               "formatted address": geo_address["results"][0]["formatted_address"],
               "location type": geo_address["results"][0]["geometry"]["location_type"]
                }

    results_keys = results.keys()

    def getter(y):
        return results[y]
    
    geocoded_row = []
    
    for x in results_keys:
        geocoded_row.append(getter(x))
    
    return geocoded_row


for row in test_csv:
    new_row = geocode_save(row)
    new_df.append(new_row)
    
new_df.head()

IndexError: list index out of range